In [2]:
# !wget https://huggingface.co/datasets/malaysia-ai/starcoderdata-sample/resolve/main/kotlin.jsonl

In [3]:
from huggingface_hub import InferenceClient
from tqdm import tqdm
import os
import json
from glob import glob

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [4]:
ls = []

with open('kotlin.jsonl') as fopen:
    for l in tqdm(fopen):
        try:
            l = json.loads(l)
            if len(l['content'].split()) > 5e3:
                continue
            ls.append(l['content'])
        except:
            pass
len(ls)

200000it [00:08, 24736.16it/s]


199571

In [5]:
client = InferenceClient(
    '', timeout = 120
)


def format_prompt(message, history):
  prompt = "<s>"
  for user_prompt, bot_response in history:
    prompt += f"[INST] {user_prompt} [/INST]"
    prompt += f" {bot_response}</s> "
  prompt += f"[INST] {message} [/INST]"
  return prompt

def format_user(history):
    prompt = "<s>"
    for user_prompt, bot_response in history:
        prompt += f"[INST] {user_prompt} [/INST]"
        prompt += f" {bot_response}</s> "
    prompt += f"[INST]"
    return prompt

In [6]:
generate_kwargs = dict(
    temperature=1.0,
    max_new_tokens=5120,
    top_p=0.95,
    repetition_penalty=1.0,
    do_sample=True,
)

In [7]:
!mkdir mixtral-kotlin
# !rm -rf mixtral-kotlin/*.json

In [12]:
def answer(q, i):
    filename = f'mixtral-kotlin/{i}.json'
    if os.path.exists(filename):
        return
    
    while True:
    # for _ in range(5):
        try:
            
            prompt = f"""
Please gain inspiration from the following random code snippet to create a high-quality Kotlin problem. Present your output in two distinct sections: [Problem Description] and [Solution].

Kotlin snippet for inspiration:
```
{q}
```

Guidelines for each section:

1. [Problem Description]: This should be **completely self-contained**, providing all the contextual information one needs to understand and solve the problem. Assume common programming knowledge, but ensure that any specific context, variables, or code snippets pertinent to this problem are explicitly included.

2. [Solution]: Offer a comprehensive, **correct** solution that accurately addresses the [Problem Description] you provided.
""".strip()
            formatted_prompt = format_prompt(prompt, [])
            stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
            output = stream.generated_text
            with open(filename, 'w') as fopen:
                json.dump(output, fopen)
            break
        except Exception as e:
            if 'tokens + `max_new_tokens`' in str(e):
                with open(filename, 'w') as fopen:
                    json.dump(False, fopen)
                break
            pass

In [9]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [10]:
from threading import Thread
from queue import Queue

queue = Queue()
questions = ls
urls = [(q, no) for no, q in enumerate(questions)]
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [15]:
answer(*urls[0])

In [16]:
with open('mixtral-kotlin/0.json') as fopen:
    print(json.loads(fopen.read()))

 **[Problem Description]**

In a Discord server, a bot can interact with users through commands, which can have various options, permissions, and other settings. The provided Kotlin code snippet contains data classes representing these objects. Your task is to create a command called `ChangeCommandOptions` that modifies the options of an existing application command in a given guild. The command should meet the following requirements:

1. Accept a command name as its first argument.
2. Accept a new list of options (type: `List<ApplicationCommandOption>`) as its second argument.
3. Check if the bot has the necessary permissions to modify the command's options in the target guild.
4. Modify the command's options using the `ApplicationCommandPermissionsEditRequest` data class.

Assume that you have access to the following dependencies:

- `dev.kord.core.Kord`: A library instance that handles bot connections and other features.
- `dev.kord.core.entity.Guild`: Represents a Discord guild.
- 

In [ ]:
max_worker = 50
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

  8%|███████▊                                                                                         | 16056/199571 [11:47:28<50:29:27,  1.01it/s]